In [11]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
#import censusdata
import re

censuskey = '7b3e8a6afe58867abcf5aecad26845d802115f1f'

In [12]:
HUDcsv = pd.read_csv('data/hudPicture2018_363148.csv')
HUDcsv.head()
clean_HUDcsv = HUDcsv.drop(HUDcsv.columns[[0, 1, 2, 3, 4, 65, 64, 63, 62, 61, 60, 59, 58]], axis=1)


list(clean_HUDcsv.columns) 


to_remove = ['Average months since report','% moved in past year','Average months on waiting list','Average months since moved in',
            '% with utility allowance','Average utility allowance $$']
clean_HUDcsv = clean_HUDcsv[clean_HUDcsv.columns.difference(to_remove)]

# only keep the related columns
cols_of_interest=['Code','Subsidized units available','% Occupied','% Reported','Household income per year','% very low income','% extremely low income'
       ,'% female head','% with disability among Head Spouse Co-head aged 61 years or less','% with disability among Head Spouse Co-head aged 62 years or older',
       '% in poverty (Census tract)','% minority (Census tract)']
clean_HUDcsv = clean_HUDcsv[cols_of_interest]
clean_HUDcsv.rename(columns={'Code': 'GEOID'},inplace=True)
clean_HUDcsv

,GEOID,Subsidized units available,% Occupied,% Reported,Household income per year,% very low income,% extremely low income,% female head,% with disability among Head Spouse Co-head aged 61 years or less,% with disability among Head Spouse Co-head aged 62 years or older,% in poverty (Census tract),% minority (Census tract)
0,13001950100,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,13001950200,70,94,100,13989,89,52,86,42,62,28,54
2,13001950300,114,90,100,13061,87,51,78,47,61,31,33
3,13001950400,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,13001950500,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,13001999999,1,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4
6,13003960100,23,96,100,10766,91,64,82,29,20,27,46
7,13003960200,47,91,100,10173,96,67,89,15,71,24,47
8,13003960300,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,13003999999,2,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4


In [40]:
clean_HUDcsv.loc[clean_HUDcsv['GEOID'] == '13121001800']

,GEOID,Subsidized units available,% Occupied,% Reported,Household income per year,% very low income,% extremely low income,% female head,% with disability among Head Spouse Co-head aged 61 years or less,% with disability among Head Spouse Co-head aged 62 years or older,% in poverty (Census tract),% minority (Census tract)
978,13121001800,482,92,97,11727,98,86,46,76,43,33,63


In [13]:
#Define Funtions

#Funcion For NonDigit Removing Characters from Row - Used for Extracting GEOID
def remove_chars(s):
    clean = int(re.sub('[^0-9]+', '', str(s))[-11:])
    return clean

#Function to get census data according to requirements
def getcensustables(table):
    tablename = 'data_' + table
    print(f'Downloading {table}')
    
    # Set requirements to pull from Census API
    
    variablestopull = allvariablescsv.loc[allvariablescsv['Group']==table]['Name'].tolist()
    listofcounties = ['121', '089']
    yearstopull = range(2015,2018,1)

    #Pull Census Data according to requirement above
    atlantadata = pd.DataFrame()

    for year in yearstopull:
        for county in listofcounties:

            newdata = censusdata.download('acs5', year, censusdata.censusgeo([('state', '13'), ('county', county), ('tract', '*')]),variablestopull,censuskey)
            type(newdata)
            newdata['YEAR'] = year
            atlantadata = atlantadata.append(newdata)

    #Rename Column Headings from Code to Text
    for variable in variablestopull:
        #print(variable)
        renamevar1 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Label'].iloc[0]
        renamevar2 = allvariablescsv.loc[allvariablescsv['Name']==variable]['Concept'].iloc[0]
        renamevar = renamevar2 + '_'+ renamevar1
        #print(renamevar)
        atlantadata = atlantadata.rename(columns={variable: renamevar})

    #Transform Index to create GeoID

    #Create GeoID Column
    atlantadata['Name2'] = atlantadata.index
    atlantadata['GEOID'] = atlantadata['Name2'].apply(remove_chars)
    atlantadata.drop(columns=['Name2'], inplace=True)
    
    #Only Keep Atlanta Census Tracts(Geocodes)
    finaldata = atlantageocodes.merge(atlantadata, how='left', on='GEOID')
    
    finaldata = finaldata.dropna(axis=0, how='any')
    
    return finaldata

In [14]:
#Code Begins here

#Tables to Load from data/tablestopull.csv
tablestoloadcsv = pd.read_csv('data/tablestopull.csv')
tablestoload = tablestoloadcsv['Tables'].tolist()

#Uncommnent Line below to override csv
#tablestoload = ['B19013','B01003','B11001', 'B25064', 'B25074','B25063', 'B25061','B25056','B00002','B25001','B25122']

#Allvariables Table to get table detail
allvariablescsv = pd.read_csv('data/allvariables.csv')

#Atlanta Geo Codes Table to filter to city of atlanta only 
atlantageocodes = pd.read_csv('data/atlantageocodes.csv')

#Create dictionary with each cleaned table as an entry
alltables = dict()  


#Script will attempt to load tables from data folder first, otherwise download them
for table in tablestoload:
    try:
        data = pd.read_csv('data/' + table + '.csv')
        alltables[table] = data
        print(f'Loading {table}')
    except:
        data = getcensustables(table)
        alltables[table] = data

        data.to_csv('data/' + table + '.csv')
        
        

Loading B19013
Loading B01003
Loading B11001
Loading B25064
Loading B25074
Loading B25063
Loading B25061
Loading B25056
Loading B00002
Loading B25001
Loading B25122


In [15]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year
percentrentbyincome = alltables['B25074']

percentrentbyincome_pd = pd.DataFrame(percentrentbyincome)
display(percentrentbyincome_pd) ###for class



percentrentbyincome = percentrentbyincome.groupby('YEAR').sum()
percentrentbyincome = percentrentbyincome.transpose()


#Split index into variables
percentrentbyincome['YEAR'] = percentrentbyincome.index
split1 = percentrentbyincome['YEAR'].str.split('PAST 12 MONTHS_Estimate!!Total!!', expand=True)
split2 = split1[1].str.split('!!', expand=True)

percentrentbyincome['Income'] = split2[0]
percentrentbyincome['Rent as % of Income'] = split2[1]

percentrentbyincome.drop(columns='YEAR')


#Set years to loop through
years = [2014.0,2015.0,2016.0,2017.0]

percentrentbyincome2 = percentrentbyincome.copy()


for year in years:

    print(year)
    percentrentbyincome = percentrentbyincome2[[year,'Income','Rent as % of Income']]

    #display(percentrentbyincome)
    percentrentbyincome = percentrentbyincome.dropna(axis=0, how='any')
    #display(percentrentbyincome)
    # display(percentrentbyincome.pivot(index=percentrentbyincome['Income'], columns=percentrentbyincome['Rent Change'], values=percentrentbyincome[2012.0]))

    percentrentbyincomepv = pd.crosstab(index=percentrentbyincome.iloc[:,1], columns=percentrentbyincome.iloc[:,2], 
                      values=percentrentbyincome.iloc[:,0], aggfunc='mean')

    percentrentbyincomepv['TotalHHs'] = percentrentbyincomepv.sum(axis = 1, skipna = True) 

    #percentrentbyincomepv = percentrentbyincomepv.div(percentrentbyincomepv.TotalHHs, axis=0)*100

    display(percentrentbyincomepv)

,Unnamed: 0,GEOID,"HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!Less than 20.0 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!20.0 to 24.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!25.0 to 29.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!30.0 to 34.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!35.0 to 39.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!40.0 to 49.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!Less than $10,000!!50.0 percent or more",...,"HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$75,000 to $99,999!!50.0 percent or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!Less than 20.0 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!20.0 to 24.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!25.0 to 29.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!30.0 to 34.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!35.0 to 39.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!40.0 to 49.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!50.0 percent or more",YEAR
0,0,13121006601,118,0,0,18,0,0,0,66,...,0,8,8,0,0,0,0,0,0,2014
1,1,13121006601,100,0,0,18,0,0,0,49,...,0,4,4,0,0,0,0,0,0,2015
2,2,13121006601,128,0,0,21,0,0,0,72,...,0,6,6,0,0,0,0,0,0,2016
3,3,13121006601,108,0,0,7,0,0,0,66,...,0,3,3,0,0,0,0,0,0,2017
4,4,13121006700,302,42,0,16,0,0,0,90,...,0,21,21,0,0,0,0,0,0,2014
5,5,13121006700,226,44,0,17,0,0,0,57,...,0,0,0,0,0,0,0,0,0,2015
6,6,13121006700,197,25,0,16,0,0,0,51,...,0,11,11,0,0,0,0,0,0,2016
7,7,13121006700,217,40,20,17,0,0,0,57,...,0,12,12,0,0,0,0,0,0,2017
8,8,13121009300,85,0,0,0,0,0,0,73,...,0,221,200,21,0,0,0,0,0,2014
9,9,13121009300,59,0,0,0,0,0,0,44,...,0,228,209,19,0,0,0,0,0,2015


2014.0


Rent as % of Income,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 to 39.9 percent,40.0 to 49.9 percent,50.0 percent or more,Less than 20.0 percent,TotalHHs
Income,,,,,,,,
"$10,000 to $19,999",630.0,1165.0,765.0,555.0,1741.0,10565.0,942.0,16363.0
"$100,000 or more",784.0,204.0,70.0,43.0,137.0,60.0,8901.0,10199.0
"$20,000 to $34,999",747.0,1905.0,2666.0,3137.0,3931.0,5108.0,848.0,18342.0
"$35,000 to $49,999",2472.0,3767.0,2849.0,1388.0,1115.0,377.0,989.0,12957.0
"$50,000 to $74,999",4854.0,2904.0,1346.0,517.0,521.0,147.0,4176.0,14465.0
"$75,000 to $99,999",1814.0,673.0,216.0,95.0,14.0,39.0,4922.0,7773.0
"Less than $10,000",321.0,1044.0,583.0,392.0,578.0,11012.0,406.0,14336.0


2015.0


Rent as % of Income,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 to 39.9 percent,40.0 to 49.9 percent,50.0 percent or more,Less than 20.0 percent,TotalHHs
Income,,,,,,,,
"$10,000 to $19,999",586.0,1117.0,728.0,578.0,1712.0,10799.0,856.0,16376.0
"$100,000 or more",801.0,285.0,73.0,64.0,124.0,52.0,9950.0,11349.0
"$20,000 to $34,999",681.0,1786.0,2758.0,3046.0,4197.0,4917.0,740.0,18125.0
"$35,000 to $49,999",2700.0,4255.0,2755.0,1593.0,1197.0,420.0,1193.0,14113.0
"$50,000 to $74,999",5113.0,3019.0,1447.0,601.0,484.0,121.0,4611.0,15396.0
"$75,000 to $99,999",1993.0,804.0,164.0,155.0,33.0,39.0,5138.0,8326.0
"Less than $10,000",328.0,1007.0,450.0,326.0,637.0,10849.0,405.0,14002.0


2016.0


Rent as % of Income,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 to 39.9 percent,40.0 to 49.9 percent,50.0 percent or more,Less than 20.0 percent,TotalHHs
Income,,,,,,,,
"$10,000 to $19,999",824.0,1193.0,800.0,625.0,1796.0,10923.0,803.0,16964.0
"$100,000 or more",1073.0,447.0,40.0,57.0,52.0,0.0,10884.0,12553.0
"$20,000 to $34,999",618.0,1970.0,2862.0,2801.0,4583.0,4945.0,865.0,18644.0
"$35,000 to $49,999",2523.0,4172.0,2960.0,1687.0,1414.0,699.0,1019.0,14474.0
"$50,000 to $74,999",5336.0,3562.0,1539.0,818.0,485.0,150.0,4389.0,16279.0
"$75,000 to $99,999",2381.0,792.0,280.0,173.0,47.0,1.0,4944.0,8618.0
"Less than $10,000",279.0,831.0,649.0,200.0,595.0,10093.0,440.0,13087.0


2017.0


Rent as % of Income,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 to 39.9 percent,40.0 to 49.9 percent,50.0 percent or more,Less than 20.0 percent,TotalHHs
Income,,,,,,,,
"$10,000 to $19,999",882.0,1241.0,806.0,468.0,1541.0,10248.0,921.0,16107.0
"$100,000 or more",1374.0,491.0,80.0,96.0,65.0,0.0,11984.0,14090.0
"$20,000 to $34,999",665.0,1637.0,2827.0,2743.0,4552.0,5058.0,972.0,18454.0
"$35,000 to $49,999",2332.0,4086.0,3172.0,1859.0,1560.0,809.0,1091.0,14909.0
"$50,000 to $74,999",5172.0,3830.0,1917.0,1159.0,624.0,169.0,4113.0,16984.0
"$75,000 to $99,999",2841.0,1116.0,350.0,200.0,52.0,12.0,4552.0,9123.0
"Less than $10,000",421.0,897.0,536.0,219.0,600.0,9194.0,400.0,12267.0


In [25]:

percentrentbyincome_pd.GEOID = percentrentbyincome_pd.GEOID.astype(str)
clean_HUDcsv.GEOID = clean_HUDcsv.GEOID.astype(str)

A=type(clean_HUDcsv.ix[0, 'GEOID'])
print(A)

B=type(percentrentbyincome_pd.ix[0, 'GEOID'])
print(B)


<class 'str'>
<class 'str'>


C:\Users\user\Anaconda3\envs\Pythondata\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\Users\user\Anaconda3\envs\Pythondata\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [24]:
pd.merge(clean_HUDcsv, percentrentbyincome_pd, on='GEOID', how='inner', right_index=True)



,GEOID,Subsidized units available,% Occupied,% Reported,Household income per year,% very low income,% extremely low income,% female head,% with disability among Head Spouse Co-head aged 61 years or less,% with disability among Head Spouse Co-head aged 62 years or older,...,"HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$75,000 to $99,999!!50.0 percent or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!Less than 20.0 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!20.0 to 24.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!25.0 to 29.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!30.0 to 34.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!35.0 to 39.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!40.0 to 49.9 percent","HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS_Estimate!!Total!!$100,000 or more!!50.0 percent or more",YEAR
624,13089020100,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,33,33,0,0,0,0,0,0,2014
624,13089020100,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,46,34,6,0,6,0,0,0,2015
624,13089020100,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,80,67,6,0,7,0,0,0,2016
624,13089020100,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,77,60,12,0,5,0,0,0,2017
625,13089020200,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,102,63,39,0,0,0,0,0,2014
625,13089020200,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,111,79,32,0,0,0,0,0,2015
625,13089020200,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,84,65,19,0,0,0,0,0,2016
625,13089020200,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,56,47,9,0,0,0,0,0,2017
626,13089020300,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,126,113,13,0,0,0,0,0,2014
626,13089020300,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,117,117,0,0,0,0,0,0,2015


In [19]:
#Pull HOUSEHOLD INCOME BY GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS and group by year

percentrentbyincome = alltables['B25074']
percentrentbyincome = percentrentbyincome.groupby('YEAR').sum() 
percentrentbyincome = percentrentbyincome.transpose()


#Split index into variables

percentrentbyincome['YEAR'] = percentrentbyincome.index
split1 = percentrentbyincome['YEAR'].str.split('PAST 12 MONTHS_Estimate!!Total!!', expand=True)
split2 = split1[1].str.split('!!', expand=True)
percentrentbyincome['Income'] = split2[0]
percentrentbyincome['Rent as % of Income'] = split2[1]
percentrentbyincome.drop(columns='YEAR')

#Set years of ACS5 to loop
years = [2015.0,2016.0,2017.0]


#Loop through the years of ACS5 to create a single organized dataframe
percentrentbyincomesum = pd.DataFrame()
percentrentbyincome2 = percentrentbyincome.copy()
percentrentbyincomesum2 = percentrentbyincomesum.copy()

for year in years:
    
    #Keep only relevant data points
    percentrentbyincome = percentrentbyincome2[[year,'Income','Rent as % of Income']]
    percentrentbyincome = percentrentbyincome.dropna(axis=0, how='any')
    
    #Create Matrix from Data
    percentrentbyincomepv = pd.crosstab(index=percentrentbyincome.iloc[:,1], columns=percentrentbyincome.iloc[:,2], 
                      values=percentrentbyincome.iloc[:,0], aggfunc='mean')
    percentrentbyincomepv['TotalHHs'] = percentrentbyincomepv.sum(axis = 1, skipna = True) 
    oldindex = percentrentbyincomepv.index.tolist()
    
    #Rebucket old census income ranges to our ranges
    percentrentbyincomepv.loc['Low Income'] = percentrentbyincomepv.loc['$10,000 to $19,999'] + percentrentbyincomepv.loc['Less than $10,000'] + percentrentbyincomepv.loc['$20,000 to $34,999']
    percentrentbyincomepv.loc['Moderate Income'] = percentrentbyincomepv.loc['$35,000 to $49,999'] + percentrentbyincomepv.loc['$50,000 to $74,999'] 
    percentrentbyincomepv.loc['High Income'] = percentrentbyincomepv.loc['$75,000 to $99,999'] + percentrentbyincomepv.loc['$100,000 or more'] 
    percentrentbyincomepv.drop(axis=0, index=oldindex, inplace=True)
    
    #Rebucket old census rent ranges to our ranges
    under30 = ['20.0 to 24.9 percent','25.0 to 29.9 percent','Less than 20.0 percent']
    over30 = ['30.0 to 34.9 percent','35.0 to 39.9 percent','40.0 to 49.9 percent', '50.0 percent or more']
    
    percentrentbyincomepv['Less than 30.0 percent'] = percentrentbyincomepv[under30].sum(axis=1)
    percentrentbyincomepv.drop(axis=1, columns=under30, inplace=True)
    
    percentrentbyincomepv['Over 30.0 percent'] = percentrentbyincomepv[over30].sum(axis=1)
    percentrentbyincomepv.drop(axis=1, columns=over30, inplace=True)
    
    #Add year column
    percentrentbyincomepv['YEAR'] = year
    
    #Append to final Dataframe
    percentrentbyincomesum = percentrentbyincomesum.append(percentrentbyincomepv)

#Reorder colums    
orderedcols = ['YEAR', 'Less than 30.0 percent', 'Over 30.0 percent', 'TotalHHs']
percentrentbyincomesum = percentrentbyincomesum.loc[:, orderedcols]



#percentrentbyincomepv = percentrentbyincomepv.div(percentrentbyincomepv.TotalHHs, axis=0)*100
#percentrentbyincomesum['Over 30% Percent'] = percentrentbyincomesum['Over 30.0 percent'].div(percentrentbyincomesum.TotalHHs, axis=0)
display(percentrentbyincomesum)

Rent as % of Income,YEAR,Less than 30.0 percent,Over 30.0 percent,TotalHHs
Income,,,,
Low Income,2015.0,7506.0,40997.0,48503.0
Moderate Income,2015.0,20891.0,8618.0,29509.0
High Income,2015.0,18971.0,704.0,19675.0
Low Income,2016.0,7823.0,40872.0,48695.0
Moderate Income,2016.0,21001.0,9752.0,30753.0
High Income,2016.0,20521.0,650.0,21171.0
Low Income,2017.0,8036.0,38792.0,46828.0
Moderate Income,2017.0,20624.0,11269.0,31893.0
High Income,2017.0,22358.0,855.0,23213.0


In [ ]:
# Generate plot for Rent as % of Income

#Settings
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

#Variables
colors = {'Less than 30.0 percent':'red', 'Over 30.0 percent':'blue'}
groups = percentrentbyincomesum.groupby('Income')
percentbuckets = ['Less than 30.0 percent', 'Over 30.0 percent']
filename = '% Rent of Income, # of Households'

#Loop through each bucket and make chart
for bucket in percentbuckets:
    fig, ax = plt.subplots(figsize=(20, 10))
    for name, group in groups:

        #print(name)

        x = ['2015','2016','2017']
        y = percentrentbyincomesum.loc[percentrentbyincomesum.index==name, bucket]
        #print(percentbucket)
        plt.plot(x, y,color=colors.get(name), label=name,linewidth=4.0)
    ax.legend()
    ax.grid()
    # Create a title, x label, and y label for our chart

    plt.title(bucket + ' ' + "Gross Rent # of Households" , fontsize=18)
    plt.xlabel("Year", fontsize=18)
    plt.ylabel("# of Households", fontsize=18)


    #Save the Figure
    plt.savefig("images/" + filename + ' ' +bucket + ".png")
    plt.show()





In [ ]:
# Generate the Plot from percentrentbyincomesum for Total Households

#Drop irrelevant data
todrop = ['Less than 30.0 percent','Over 30.0 percent']
hhsbyincome = percentrentbyincomesum.drop(axis=1, columns=todrop)


# Settings
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)



#Variables
colors = {'TotalHHs':'blue', '30.0 to 34.9 percent':'blue', '35.0 to 39.9 percent':'green', '40.0 to 49.9 percent':'black', '50.0 percent or more':'pink'}
groups = hhsbyincome.groupby('Income')
filename = 'Total Households'

#Loop through all buckets
percentbuckets = ['TotalHHs']
for bucket in percentbuckets:
    fig, ax = plt.subplots(figsize=(20, 10))
    for name, group in groups:

        #print(name)

        x = ['2015','2016','2017']
        y = percentrentbyincomesum.loc[percentrentbyincomesum.index==name, bucket]
        #print(percentbucket)
        plt.plot(x, y,color=colors.get(name), label=name,linewidth=4.0)
    ax.legend()
    ax.grid()
        # Create a title, x label, and y label for our chart

    plt.title(bucket, fontsize=18)
    plt.xlabel("Year", fontsize=18)
    plt.ylabel("# of Households", fontsize=18)
    

    # Save the Figure
    plt.savefig("images/" + filename + ".png")
    
    #Display chart
    plt.show()




In [ ]:
#Get Occupied Units by Rent Paid tables

occupiedunits = alltables['B25056']

#Group by year for simplicity
occupiedunits = occupiedunits.groupby('YEAR').sum()
occupiedunits = occupiedunits.transpose()


#Split index into relevant variables
occupiedunits['YEAR'] = occupiedunits.index
split1 = occupiedunits['YEAR'].str.split('!!', expand=True)
occupiedunits['Rent'] = split1[3]

#Backuptable for later
occupiedunitsog = occupiedunits.copy()

#Keep relevant results
occupiedunits.index = occupiedunits['Rent']
occupiedunits.drop(axis=1,columns=['Rent','YEAR'], inplace=True)
occupiedunits = occupiedunits.iloc[3:]

#Display table
display(occupiedunits)




In [ ]:
#Get Unoccupied Units by Rent Paid tables

unoccupiedunits = alltables['B25061']

#Aggregate table for simplicity
unoccupiedunits = unoccupiedunits.groupby('YEAR').sum()
unoccupiedunits = unoccupiedunits.transpose()

#Split index into relevant variables
unoccupiedunits['YEAR'] = unoccupiedunits.index
split1 = unoccupiedunits['YEAR'].str.split('!!', expand=True)
unoccupiedunits['Rent'] = split1[2]

#Keep relevant results
unoccupiedunits.index = unoccupiedunits['Rent']
unoccupiedunits.drop(axis=1,columns=['Rent','YEAR'], inplace=True)
unoccupiedunits = unoccupiedunits.iloc[3:]

#Display Table
display(unoccupiedunits)

display(unoccupiedunits[2017.0].sum())

In [ ]:
#Here we will take Occupied + Unoccupied rental Units to determine supply available
occupiedunits.dropna(inplace=True)
unoccupiedunits.dropna(inplace=True)


#Join both tables
allunits = occupiedunits.join(unoccupiedunits, how='outer',lsuffix=' Occupied Units', rsuffix=' Unoccupied Units')

#Clean up result
allunits = allunits.fillna(0)
allunits.dropna(inplace=True)


#Years to loop through
years = [2015.0,2016.0,2017.0]


#Loop through each year to organize data into one table
allunitsonly = pd.DataFrame()
for year in years:
    year = str(year)
    allunitsonly['All Units ' + year] = allunits[year + ' Unoccupied Units'] +allunits[year + ' Occupied Units']

 
####display(allunitsonly)
####display(unoccupiedunits.index.tolist())

#####allunitsonly = allunitsonly.reindex(unoccupiedunits.index.tolist())


allunitsonly.dropna(inplace=True)


#Need to sum results in each rental bracket
list0 =[0]*len(allunitsonly.columns)


#Define rent to income brackets
allunitsonly.loc['Rental Units Available to Low Income HHs'] = list0
allunitsonly.loc['Rental Units Available to Med Income HHs'] = list0
allunitsonly.loc['Rental Units Available to High Income HHs'] = list0


#Define Rental brackets
lowincomerent = ['Less than $100', '$100 to $149', '$150 to $199', '$200 to $249', '$250 to $299', '$300 to $349', '$350 to $399', '$400 to $449', '$450 to $499', '$500 to $549', '$550 to $599', '$600 to $649', '$650 to $699', '$700 to $749', '$750 to $799']
medincomerent = ['$800 to $899','$900 to $999', '$1,000 to $1,249', '$1,250 to $1,499', '$1,500 to $1,999']
highincomerent = ['$2,000 to $2,499', '$2,500 to $2,999', '$3,000 to $3,499', '$3,500 or more']

#Generate new rental bracekts
for low in lowincomerent:
    allunitsonly.loc['Rental Units Available to Low Income HHs'] += allunitsonly.loc[low]
    
for med in medincomerent:
    allunitsonly.loc['Rental Units Available to Med Income HHs'] += allunitsonly.loc[med]
    
for high in highincomerent:
    allunitsonly.loc['Rental Units Available to High Income HHs'] += allunitsonly.loc[high]

#Keep only the summarized rental brackets
allunitsonlysum = allunitsonly.tail(3)


print('Affordable Supply in each bracket')
display(allunitsonlysum)




In [ ]:
# Generate the Plot from percentrentbyincomesum for Total Households

# #Drop irrelevant data
# todrop = ['Less than 30.0 percent','Over 30.0 percent']
# hhsbyincome = percentrentbyincomesum.drop(axis=1, columns=todrop)


# Settings
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)



#Variables
colors = {'All Units 2015.0':'blue', 'All Units 2016.0':'blue', 'All Units 2017.0':'green'}
filename = 'Rental Units Available'
fig, ax = plt.subplots(figsize=(20, 10))
#Loop through all buckets
unitbuckets = allunitsonlysum.index.tolist()
for bucket in unitbuckets:

    print(bucket)

    y = allunitsonlysum.loc[bucket]
    x = ['2015','2016','2017']
    #print(percentbucket)
    plt.plot(x, y,color=colors.get(bucket), label=bucket,linewidth=4.0)

ax.legend()
ax.grid()
    # Create a title, x label, and y label for our chart

plt.title('Rental Units Available', fontsize=18)
plt.xlabel("Year", fontsize=18)
plt.ylabel("# of Households", fontsize=18)


# Save the Figure
plt.savefig("images/" + filename + ".png")

#Display chart
plt.show()




In [ ]:
print('Demand in each bracket')
display(hhsbyincome)